In [1]:
! start .

In [14]:
! code .

In [3]:
import os,git,shutil
import pandas as pd

# 指定されたデータサイズでファイルを分割する
def divide_file(filePath, chunkSize):
    readedDataSize = 0
    i = 0
    fileList = []
    # 対象ファイルを開く
    f = open(filePath, "rb")
    # ファイルを読み終わるまで繰り返す
    contentLength = os.path.getsize(filePath)
    while readedDataSize < contentLength:
        # 読み取り位置をシーク
        f.seek(readedDataSize)
        # 指定されたデータサイズだけ読み込む
        data = f.read(chunkSize)
        # 分割ファイルを保存
        saveFilePath = filePath + "." + str(i)
        with open(saveFilePath, 'wb') as saveFile:
            saveFile.write(data)
        # 読み込んだデータサイズの更新
        readedDataSize = readedDataSize + len(data)
        i = i + 1
        fileList.append(saveFilePath)
    return fileList

# 渡されたファイルリストの順序で１つのファイルに結合する
def join_file(fileList, filePath):
    with open(filePath, 'wb') as saveFile:
        for f in fileList:
            data = open(f, "rb").read()
            saveFile.write(data)
            saveFile.flush()
            
class env_util():
    def __init__(self):
        self.u_path = os.environ["userprofile"]
        self.miniconda_path = os.path.join(self.u_path,"miniconda3","pkgs")
        self.miniconda_path = os.path.join(self.u_path,"miniconda3","pkgs")
        self.freecad_path = os.path.join(self.u_path,"AppData","Roaming","FreeCAD")
        self.fac_path = os.path.join(self.u_path,"000_work","factory_env_miniconda39")
        self.fac_pkg_path = os.path.join(self.fac_path,"pkgs")
        self.repo_fac = git.Repo(self.fac_path)
    #         self.repo_freecad = git.Repo(self.freecad_path)
        self.remote_folder_path = os.path.join(self.u_path,"000_work","env_miniconda_39")
        self.remote_pkg_path = os.path.join(self.remote_folder_path,"pkgs")
        self.repo_remote_folder = git.Repo(self.remote_folder_path)

    def get_status_text(self,case):
        if case == 0:
            tex = self.repo_fac.git.status("-s")
        elif case == 1:
            tex = self.repo_remote_folder.git.status("-s")
#         elif case == 2:
#             tex = self.repo_freecad.git.status("-s")
        return tex

    def comfirm_file(self,case):
        tex = self.get_status_text(case)
        if tex.find("\n")  != 0:
            li = tex.split("\n")
            print(li)

        return li

    def get_list(self):
        file_list = []
        file_list_batsu = []
        for root, dirs, files in os.walk(self.miniconda_path):
            for filename in files:
                fname = os.path.join(root, filename)
                file_list.append([fname,os.path.getsize(fname) / 1000000])
            df = pd.DataFrame(file_list)
            df.columns=["path","MB"]
        return df

if __name__ == '__main__':
    self = env_util()

# おうち側作業

リポジトリの中に未コミットがあれば、コミットとして0にする

In [15]:
li = self.comfirm_file(case=1)
li

[' ? diff-dwg', ' D gantt/gantt-howto-node', ' M utility.ipynb', ' M utility.py', '?? gantt/gantt-howto-node_s/']


[' ? diff-dwg',
 ' D gantt/gantt-howto-node',
 ' M utility.ipynb',
 ' M utility.py',
 '?? gantt/gantt-howto-node_s/']

In [16]:
if len(li) != 0:
    self.repo_remote_folder.git.add(r"*")
    li = self.comfirm_file(case=1)

[' ? diff-dwg', 'D  gantt/gantt-howto-node', 'A  gantt/gantt-howto-node_s/README.md', 'A  gantt/gantt-howto-node_s/node_modules/.bin/mime', 'A  gantt/gantt-howto-node_s/node_modules/.bin/mime.cmd', 'A  gantt/gantt-howto-node_s/node_modules/.bin/mime.ps1', 'A  gantt/gantt-howto-node_s/node_modules/.package-lock.json', 'A  gantt/gantt-howto-node_s/node_modules/@types/bluebird/LICENSE', 'A  gantt/gantt-howto-node_s/node_modules/@types/bluebird/README.md', 'A  gantt/gantt-howto-node_s/node_modules/@types/bluebird/index.d.ts', 'A  gantt/gantt-howto-node_s/node_modules/@types/bluebird/package.json', 'A  gantt/gantt-howto-node_s/node_modules/@types/bluebird/ts4.1/index.d.ts', 'A  gantt/gantt-howto-node_s/node_modules/@types/mysql/LICENSE', 'A  gantt/gantt-howto-node_s/node_modules/@types/mysql/README.md', 'A  gantt/gantt-howto-node_s/node_modules/@types/mysql/index.d.ts', 'A  gantt/gantt-howto-node_s/node_modules/@types/mysql/package.json', 'A  gantt/gantt-howto-node_s/node_modules/@types/nod

In [17]:
if len(li) != 0:
    self.repo_remote_folder.git.commit("-m","gantt_s")

## nmp

In [4]:
os.environ["userprofile"]
f_path = os.path.join(os.environ["userprofile"],"000_work","env_miniconda_39","nmp_pkgs")
f_path

'C:\\Users\\M151325\\000_work\\env_miniconda_39\\nmp_pkgs'

['']


## pip

In [1]:
import os

In [2]:
os.environ["userprofile"]

'C:\\Users\\M151325'

In [3]:
f_path = os.path.join(os.environ["userprofile"],"000_work","env_miniconda_39","pip_pkgs")
f_path

'C:\\Users\\M151325\\000_work\\env_miniconda_39\\pip_pkgs'

In [4]:
pkg_name = "python-gantt-csv"
# print(f"pip download -d {f_path} --no-binary :all: {pkg_name}")
print(f"pip download -d {f_path} {pkg_name}")

pip download -d C:\Users\M151325\000_work\env_miniconda_39\pip_pkgs python-gantt


In [8]:
! start .

## 初期化

### freecad

ファイルをコピー

In [10]:
import pandas as pd
li_m = self.comfirm_file(case=2)
li_s = [[i[0:2],i[3:]] for i in li_m]
df = pd.DataFrame(li_s,columns=["code","file_name"])
df_tg = df[df["code"]=="??"]
li = list(df_tg["file_name"])

['']


In [11]:
li_m

['']

In [14]:
file_list = li
out_list = []
for file in file_list:
    f_path_moto = os.path.join(self.freecad_path,file)
    f_path_copy = os.path.join(self.remote_folder_path,"FreeCAD",file)
    out_list.append([f_path_copy,os.path.getsize(f_path_moto) / 1000000])
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{f_path_moto}")
        pass
df = pd.DataFrame(out_list)
if len(df) != 0:
    df.columns=["path","MB"]

100M以上は分割（factory_env_miniconda39）

In [21]:
files = os.listdir(os.path.join(self.remote_folder_path,"FreeCAD"))
file_list = [os.path.join(self.remote_folder_path,"FreeCAD",f) for f in files ]
file_list = [[f,os.path.getsize(f) / 1000000] for f in file_list ]
df = pd.DataFrame(file_list)
df.columns=["path","MB"]

In [22]:
df_b = df[df["MB"]>99]
df_b

,path,MB


In [16]:
if len(df_b)!=0:
    for target in list(df_b["path"]):
        fileList = divide_file(target, 50*10**6)
        df_a = pd.DataFrame(fileList,columns=["path"])
        df_a.to_csv(os.path.join("sep_file_list_freecad/",os.path.basename(target)+".csv"),index=False)
        os.remove(target)

### env_miniconda_39

pkgsとsep_file_listの中のファイルを削除

In [4]:
shutil.rmtree(self.remote_pkg_path)
os.mkdir(self.remote_pkg_path)
shutil.rmtree(os.path.join(self.remote_folder_path,"sep_file_list"))
os.mkdir(os.path.join(self.remote_folder_path,"sep_file_list"))

### factory_env_miniconda39

未コミットはコミットしておく

In [8]:
li = self.comfirm_file(case=0)

[' M pkgs/urls.txt', '?? pkgs/cached-property-1.5.2-hd8ed1ab_1.tar.bz2', '?? pkgs/cached_property-1.5.2-pyha770c72_1.tar.bz2', '?? pkgs/conda-4.11.0-py39hcbf5309_0.tar.bz2', '?? pkgs/isodate-0.6.0-py_1.tar.bz2', '?? pkgs/requests-file-1.5.1-pyh9f0ad1d_0.tar.bz2', '?? pkgs/requests-toolbelt-0.9.1-py_0.tar.bz2', '?? pkgs/zeep-4.1.0-pyh6c4a22f_0.tar.bz2']


未コミットがあれば、コミットとして0にする

In [9]:
if len(li) != 0:
    self.repo_fac.git.add(r"*")
    li = self.comfirm_file(case=0)

if len(li) != 0:
    self.repo_fac.git.commit("-m","initial commit")

['A  pkgs/cached-property-1.5.2-hd8ed1ab_1.tar.bz2', 'A  pkgs/cached_property-1.5.2-pyha770c72_1.tar.bz2', 'A  pkgs/conda-4.11.0-py39hcbf5309_0.tar.bz2', 'A  pkgs/isodate-0.6.0-py_1.tar.bz2', 'A  pkgs/requests-file-1.5.1-pyh9f0ad1d_0.tar.bz2', 'A  pkgs/requests-toolbelt-0.9.1-py_0.tar.bz2', 'M  pkgs/urls.txt', 'A  pkgs/zeep-4.1.0-pyh6c4a22f_0.tar.bz2']


## miniconda→factory_env_miniconda39にコピー
同じ状態にします

In [9]:
files = os.listdir(self.miniconda_path)
file_list = [f for f in files if os.path.isfile(os.path.join(self.miniconda_path, f))]
for file in file_list:
    f_path_moto = os.path.join(self.miniconda_path,file)
    f_path_copy = os.path.join(self.fac_pkg_path,file)
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{file}")
        pass

In [10]:
file_list

['.cio-config.py',
 'alabaster-0.7.12-py_0.tar.bz2',
 'altgraph-0.17-pyhd8ed1ab_0.tar.bz2',
 'ansi2html-1.6.0-py39hcbf5309_1.tar.bz2',
 'anyio-3.3.4-py39hcbf5309_0.tar.bz2',
 'appdirs-1.4.4-pyh9f0ad1d_0.tar.bz2',
 'argon2-cffi-20.1.0-py39hb82d6ee_2.tar.bz2',
 'asgiref-3.4.1-pyhd8ed1ab_0.tar.bz2',
 'astroid-2.8.0-py39hcbf5309_0.tar.bz2',
 'async_generator-1.10-py_0.tar.bz2',
 'attrs-21.2.0-pyhd8ed1ab_0.tar.bz2',
 'autopep8-1.5.7-pyhd8ed1ab_0.tar.bz2',
 'babel-2.9.1-pyh44b312d_0.tar.bz2',
 'backcall-0.2.0-pyh9f0ad1d_0.tar.bz2',
 'backports-1.0-py_2.tar.bz2',
 'backports.functools_lru_cache-1.6.4-pyhd8ed1ab_0.tar.bz2',
 'beautifulsoup4-4.10.0-pyha770c72_0.tar.bz2',
 'blas-1.0-mkl.conda',
 'bleach-4.1.0-pyhd8ed1ab_0.tar.bz2',
 'boost-cpp-1.74.0-h5b4e17d_4.tar.bz2',
 'brotli-python-1.0.9-py39h415ef7b_5.tar.bz2',
 'brotlipy-0.7.0-py39h2bbff1b_1003.conda',
 'bzip2-1.0.8-h8ffe710_4.tar.bz2',
 'ca-certificates-2021.10.8-h5b45459_0.tar.bz2',
 'ca-certificates-2021.5.30-h5b45459_0.tar.bz2',
 'ca-

## factory_env_miniconda39→env_miniconda_39にコピー
factory_env_miniconda39の未コミットのファイルをenv_miniconda_39にコピーします

In [11]:
import pandas as pd
li_m = self.comfirm_file(case=0)
li_s = [[i[0:2],i[3:]] for i in li_m]
df = pd.DataFrame(li_s,columns=["code","file_name"])
df_tg = df[df["code"]=="??"]
li = list(df_tg["file_name"])

[' M pkgs/urls.txt', '?? pkgs/cached-property-1.5.2-hd8ed1ab_1.tar.bz2', '?? pkgs/cached_property-1.5.2-pyha770c72_1.tar.bz2', '?? pkgs/conda-4.11.0-py39hcbf5309_0.tar.bz2', '?? pkgs/isodate-0.6.0-py_1.tar.bz2', '?? pkgs/requests-file-1.5.1-pyh9f0ad1d_0.tar.bz2', '?? pkgs/requests-toolbelt-0.9.1-py_0.tar.bz2', '?? pkgs/zeep-4.1.0-pyh6c4a22f_0.tar.bz2']


In [12]:
li_m

[' M pkgs/urls.txt',
 '?? pkgs/cached-property-1.5.2-hd8ed1ab_1.tar.bz2',
 '?? pkgs/cached_property-1.5.2-pyha770c72_1.tar.bz2',
 '?? pkgs/conda-4.11.0-py39hcbf5309_0.tar.bz2',
 '?? pkgs/isodate-0.6.0-py_1.tar.bz2',
 '?? pkgs/requests-file-1.5.1-pyh9f0ad1d_0.tar.bz2',
 '?? pkgs/requests-toolbelt-0.9.1-py_0.tar.bz2',
 '?? pkgs/zeep-4.1.0-pyh6c4a22f_0.tar.bz2']

In [13]:
file_list = li
out_list = []
for file in file_list:
    f_path_moto = os.path.join(self.fac_path,file)
    f_path_copy = os.path.join(self.remote_folder_path,file)
    out_list.append([f_path_copy,os.path.getsize(f_path_moto) / 1000000])
    try:
        shutil.copyfile(f_path_moto,f_path_copy)
    except :
        print(f"エラー:{f_path_moto}")
        pass
df = pd.DataFrame(out_list)
if len(df) != 0:
    df.columns=["path","MB"]

## 100M以上は分割（factory_env_miniconda39）

In [8]:
files = os.listdir(self.remote_pkg_path)
file_list = [os.path.join(self.remote_pkg_path,f) for f in files ]
file_list = [[f,os.path.getsize(f) / 1000000] for f in file_list ]
df = pd.DataFrame(file_list)
df.columns=["path","MB"]

SyntaxError: invalid syntax (Temp/ipykernel_17016/4011051060.py, line 1)

In [15]:
df_b = df[df["MB"]>99]
df_b

,path,MB


In [16]:
if len(df_b)!=0:
    for target in list(df_b["path"]):
        fileList = divide_file(target, 50*10**6)
        df_a = pd.DataFrame(fileList,columns=["path"])
        df_a.to_csv(os.path.join("sep_file_list/",os.path.basename(target)+".csv"),index=False)
        os.remove(target)

In [17]:
! start .

## react

In [7]:
import os
import pandas as pd

dir_path = r"C:\Users\M151325\000_work\env_miniconda_39\react"
file_list=[]
for (root, directories, files) in os.walk(dir_path):
#     for d in directories:
#         d_path = os.path.join(root, d)
#         print(d_path)
    for file in files:
        file_path = os.path.join(root, file)
#         print(file_path)
        file_list.append(file_path)

In [8]:
file_list = [[f,os.path.getsize(f) / 1000000] for f in file_list ]
df = pd.DataFrame(file_list)
df.columns=["path","MB"]

In [10]:
df_b = df[df["MB"]>99]
df_b

,path,MB


## githubへpush(factory_env_miniconda39)

In [13]:
li_m = self.comfirm_file(case=1)
li_s = [[i[0:2],i[3:]] for i in li_m]
df = pd.DataFrame(li_s,columns=["code","file_name"])
df

[' ? diff-dwg', ' M utility.ipynb', '?? react/', '?? utility.py']


,code,file_name
0,?,diff-dwg
1,M,utility.ipynb
2,??,react/
3,??,utility.py


In [14]:
if len(li_m) != 0:
    self.repo_remote_folder.git.add(r"*")
    li_mi= self.comfirm_file(case=1)

[' ? diff-dwg', 'A  react/react-gantt-demo/.gitignore', 'A  react/react-gantt-demo/README.md', 'A  react/react-gantt-demo/package-lock.json', 'A  react/react-gantt-demo/package.json', 'A  react/react-gantt-demo/public/favicon.ico', 'A  react/react-gantt-demo/public/index.html', 'A  react/react-gantt-demo/public/logo192.png', 'A  react/react-gantt-demo/public/logo512.png', 'A  react/react-gantt-demo/public/manifest.json', 'A  react/react-gantt-demo/public/robots.txt', 'A  react/react-gantt-demo/src/App.css', 'A  react/react-gantt-demo/src/App.js', 'A  react/react-gantt-demo/src/App.test.js', 'A  react/react-gantt-demo/src/components/Gantt/Gantt.css', 'A  react/react-gantt-demo/src/components/Gantt/Gantt.js', 'A  react/react-gantt-demo/src/components/Gantt/index.js', 'A  react/react-gantt-demo/src/components/MessageArea/MessageArea.css', 'A  react/react-gantt-demo/src/components/MessageArea/MessageArea.js', 'A  react/react-gantt-demo/src/components/MessageArea/index.js', 'A  react/react-

In [15]:
li_mi

[' ? diff-dwg',
 'A  react/react-gantt-demo/.gitignore',
 'A  react/react-gantt-demo/README.md',
 'A  react/react-gantt-demo/package-lock.json',
 'A  react/react-gantt-demo/package.json',
 'A  react/react-gantt-demo/public/favicon.ico',
 'A  react/react-gantt-demo/public/index.html',
 'A  react/react-gantt-demo/public/logo192.png',
 'A  react/react-gantt-demo/public/logo512.png',
 'A  react/react-gantt-demo/public/manifest.json',
 'A  react/react-gantt-demo/public/robots.txt',
 'A  react/react-gantt-demo/src/App.css',
 'A  react/react-gantt-demo/src/App.js',
 'A  react/react-gantt-demo/src/App.test.js',
 'A  react/react-gantt-demo/src/components/Gantt/Gantt.css',
 'A  react/react-gantt-demo/src/components/Gantt/Gantt.js',
 'A  react/react-gantt-demo/src/components/Gantt/index.js',
 'A  react/react-gantt-demo/src/components/MessageArea/MessageArea.css',
 'A  react/react-gantt-demo/src/components/MessageArea/MessageArea.js',
 'A  react/react-gantt-demo/src/components/MessageArea/index.js

In [16]:
if len(li_mi) != 0:
    self.repo_remote_folder.git.commit("-m","zeep")

push

In [18]:
self.repo_remote_folder.git.push()

''

# 相手側作業

リポジトリをダウンロードして、000_workの下に移動する

https://github.com/nyanko111777/env_miniconda_39/archive/refs/heads/main.zip


## 100M以上を復活

In [4]:
import pandas as pd
import os

file_list =  os.listdir("sep_file_list")
for file in file_list:
    df_temp = pd.read_csv("sep_file_list/"+file)
    fileList = list(list(df_temp["path"]))
    target = fileList[0][:fileList[0].find(".0")]
    join_file(fileList, target)
    for d_file in fileList:
        os.remove(d_file)

['mkl-2020.4-hb70f87d_311.tar.bz2.csv', 'qt-5.12.9-h5909a2a_4.tar.bz2.csv']

## インストール

- conda の場合
  `for /f %i in ('dir /A-d /B *') do conda install %i`



- pip の場合
  `for /f %i in ('dir /A-d /B *') do pip install %i`
